In [ ]:
! /opt/bin/nvidia-smi

Mon Feb 22 05:58:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Load Data From Kaggle**

In [ ]:
def import_data_from_drive(id):
  !wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='{id} -O- \
  | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt && wget --content-disposition --load-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='{id}'&confirm='$(<confirm.txt)

# import kaggle.json
import_data_from_drive('...')

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!pip install kaggle
!chmod 600 /root/.kaggle/kaggle.json

--2021-02-22 05:58:42--  https://docs.google.com/uc?export=download&id=1u-ZZKK8SiG_yyEZuylpn_hNg7CvE91a6
Resolving docs.google.com (docs.google.com)... 108.177.111.138, 108.177.111.102, 108.177.111.139, ...
Connecting to docs.google.com (docs.google.com)|108.177.111.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-7c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jiht0g4rb3cnud7a72te16f52s019g3p/1613973450000/02707175995128734220/*/1u-ZZKK8SiG_yyEZuylpn_hNg7CvE91a6?e=download [following]
--2021-02-22 05:58:43--  https://doc-00-7c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jiht0g4rb3cnud7a72te16f52s019g3p/1613973450000/02707175995128734220/*/1u-ZZKK8SiG_yyEZuylpn_hNg7CvE91a6?e=download
Resolving doc-00-7c-docs.googleusercontent.com (doc-00-7c-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connecting to doc-00-7c-docs.googleusercontent.com (doc-00

In [ ]:
!kaggle competitions download -c iamthebestcoderopen2020

 95% 233M/245M [00:03<00:00, 104MB/s] 
100% 245M/245M [00:03<00:00, 73.8MB/s]
  0% 0.00/1.85M [00:00<?, ?B/s]
100% 1.85M/1.85M [00:00<00:00, 160MB/s]
  0% 0.00/573k [00:00<?, ?B/s]
100% 573k/573k [00:00<00:00, 180MB/s]
 81% 41.0M/50.8M [00:01<00:00, 26.4MB/s]
100% 50.8M/50.8M [00:01<00:00, 42.1MB/s]
  0% 0.00/1.51M [00:00<?, ?B/s]
100% 1.51M/1.51M [00:00<00:00, 222MB/s]


In [ ]:
#unzip
!unzip /content/login.csv.zip
!unzip /content/purchase_detail.csv.zip
!unzip /content/submission.csv.zip
!unzip /content/user_info.csv.zip
!unzip /content/user_label_train.csv.zip

Archive:  /content/login.csv.zip
  inflating: login.csv               
Archive:  /content/purchase_detail.csv.zip
  inflating: purchase_detail.csv     
unzip:  cannot find or open /content/submission.csv.zip, /content/submission.csv.zip.zip or /content/submission.csv.zip.ZIP.
Archive:  /content/user_info.csv.zip
  inflating: user_info.csv           
Archive:  /content/user_label_train.csv.zip
  inflating: user_label_train.csv    


# Profile

## Data initial

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics

In [ ]:
from scipy import stats
def df_summary(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary = summary.rename(columns={"index": "Name", "B": "c"})
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    df_cols = summary['Name'].unique()
    for name in df_cols:
      summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),5) 

    return summary

In [ ]:
login = pd.read_csv('/content/login.csv')
purchase_detail = pd.read_csv('/content/purchase_detail.csv')
submission = pd.read_csv('/content/submission.csv')
user_info = pd.read_csv('/content/user_info.csv')
user_label_train = pd.read_csv('/content/user_label_train.csv')

# Feature Engineer

### Find main/sub df

In [ ]:
user_label_train.label = user_label_train.label.astype('int8')
df_main = pd.concat([user_label_train,submission], axis=0)
df_sub1 = login
df_sub2 = purchase_detail
df_sub3 = user_info

In [ ]:
#--login--
#userid: Encoded user ID
#date: login date
#login_times: total login time in date

df_sub1.head(3)

,userid,date,login_times
0,161097,2020-07-29,2
1,243570,2020-07-29,2
2,355497,2020-07-29,4


In [ ]:
#--purchase_detail--

#userid: Encoded user ID
#grass_date: order transaction date
#category_encoded: product category, encoded with number 1~23
#Order_count: total order count grouped by userid, grass_date and category_encoded (each order could have several amounts)
#Total_amount: total purchase amount grouped by userid, grass_date and category_encoded

df_sub2.head(3)

,userid,grass_date,order_count,total_amount,category_encoded
0,295790,2020-05-13,4,4,8
1,428982,2020-05-13,1,2,14
2,151685,2020-05-13,2,2,12


In [ ]:
#--user_info--

#userid: Encoded user ID
#gender: 1 for Male, 2 for Female
#is_seller: 1, if user shop available; 0, no shop available
#birth_year: user birth year
#enroll_time: enroll date in shopee

df_sub3.head(3)

,userid,gender,is_seller,birth_year,enroll_time
0,1,2.0,1,1985.0,2015-05-27
1,2,1.0,1,1980.0,2015-05-27
2,3,2.0,1,2014.0,2015-05-27


### df_sub1 - login

In [ ]:
#計算一個userid登入總次數 ＆ 登入筆數(天數概念)
g = df_sub1.groupby('userid').agg({'login_times':['sum','count']})
g.columns = ['login_times_sum','login_times_count']

In [ ]:
g.head(3)
# 1號登入50次在36天內
# userid 是index name 不是column name

,login_times_sum,login_times_count
userid,,
1,50,36
2,487,182
3,770,182


In [ ]:
df_sub1 = df_sub1.merge(g, how = 'left', left_on='userid', right_on='userid')

In [ ]:
df_sub1['login_times_avg'] = (df_sub1['login_times_sum'] / df_sub1['login_times_count'])

In [ ]:
df_sub1.head(3)

,userid,date,login_times,login_times_sum,login_times_count,login_times_avg
0,161097,2020-07-29,2,215,128,1.679688
1,243570,2020-07-29,2,88,78,1.128205
2,355497,2020-07-29,4,171,110,1.554545


In [ ]:
#user login unique table
df_sub1_unique = df_sub1[['userid','login_times_sum','login_times_count','login_times_avg']].drop_duplicates()

In [ ]:
# 總次數/天數
df_sub1_unique.head(3)

,userid,login_times_sum,login_times_count,login_times_avg
0,161097,215,128,1.679688
1,243570,88,78,1.128205
2,355497,171,110,1.554545


### df_sub2 - purchase_detail

In [ ]:
df_sub2 = purchase_detail

In [ ]:
#userid: Encoded user ID
#grass_date: order transaction date
#category_encoded: product category, encoded with number 1~23
#Order_count: total order count grouped by userid, grass_date and category_encoded (each order could have several amounts)
#Total_amount: total purchase amount grouped by userid, grass_date and category_encoded

df_sub2.head(3)

,userid,grass_date,order_count,total_amount,category_encoded
0,295790,2020-05-13,4,4,8
1,428982,2020-05-13,1,2,14
2,151685,2020-05-13,2,2,12


In [ ]:
g = df_sub2.groupby(['userid']).agg({'grass_date': ['count'], #登入日期有幾天 
                                     'order_count': ['sum'],  #Order_count為計算該同user&日期&類別下,購買的次數->加總為看該user總order數
                                     'total_amount': ['sum'], #總消費金額
                                     'category_encoded': ['nunique']}) #總消費類別distinct個數
g.columns = ['purchase_num','order_count_sum','total_amount_sum','category_encoded_nunique']
g.head(5)

,purchase_num,order_count_sum,total_amount_sum,category_encoded_nunique
userid,,,,
1,10,30,246,4
2,18,25,59,8
3,7,14,14,4
4,35,54,82,10
5,65,86,2734,15


In [ ]:
df_sub2 = df_sub2.merge(g, how='left', left_on = 'userid', right_on = 'userid')

In [ ]:
df_sub2.head(3)

,userid,grass_date,order_count,total_amount,category_encoded,purchase_num,order_count_sum,total_amount_sum,category_encoded_nunique
0,295790,2020-05-13,4,4,8,26,97,163,8
1,428982,2020-05-13,1,2,14,66,86,106,12
2,151685,2020-05-13,2,2,12,44,70,124,15


In [ ]:
df_sub2.shape

(7792956, 9)

In [ ]:
max(df_sub2['category_encoded'])

23

#### 1.One Hot Smoothing

In [ ]:
#create table as userid(7792956) * unique(category_encoded)(23)
index_one_hot = pd.get_dummies(df_sub2['category_encoded']).add_prefix('category_encoded_')
index_one_hot

,category_encoded_1,category_encoded_2,category_encoded_3,category_encoded_4,category_encoded_5,category_encoded_6,category_encoded_7,category_encoded_8,category_encoded_9,category_encoded_10,category_encoded_11,category_encoded_12,category_encoded_13,category_encoded_14,category_encoded_15,category_encoded_16,category_encoded_17,category_encoded_18,category_encoded_19,category_encoded_20,category_encoded_21,category_encoded_22,category_encoded_23
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7792951,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7792952,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
7792953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
7792954,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
df_sub2[['userid']]

,userid
0,295790
1,428982
2,151685
3,323635
4,118610
...,...
7792951,275949
7792952,255131
7792953,319036
7792954,511880


In [ ]:
#index join
df_sub2[['userid']].join(index_one_hot)

,userid,category_encoded_1,category_encoded_2,category_encoded_3,category_encoded_4,category_encoded_5,category_encoded_6,category_encoded_7,category_encoded_8,category_encoded_9,category_encoded_10,category_encoded_11,category_encoded_12,category_encoded_13,category_encoded_14,category_encoded_15,category_encoded_16,category_encoded_17,category_encoded_18,category_encoded_19,category_encoded_20,category_encoded_21,category_encoded_22,category_encoded_23
0,295790,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,428982,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,151685,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,323635,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,118610,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7792951,275949,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7792952,255131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
7792953,319036,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
7792954,511880,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
##index join & sum category by each user
key_one_hot = df_sub2[['userid']].join(index_one_hot).groupby('userid').sum()

In [ ]:
key_one_hot

,category_encoded_1,category_encoded_2,category_encoded_3,category_encoded_4,category_encoded_5,category_encoded_6,category_encoded_7,category_encoded_8,category_encoded_9,category_encoded_10,category_encoded_11,category_encoded_12,category_encoded_13,category_encoded_14,category_encoded_15,category_encoded_16,category_encoded_17,category_encoded_18,category_encoded_19,category_encoded_20,category_encoded_21,category_encoded_22,category_encoded_23
userid,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,1,0,0,3,0,1,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0
2,0,2,0,0,0,2,0,9,1,0,0,0,0,0,1,1,0,1,0,1,0,0,0
3,0,0,0,0,0,1,0,3,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0
4,0,0,3,0,0,1,0,9,0,1,2,8,0,0,1,0,0,2,0,0,0,6,2
5,2,1,3,6,3,4,0,9,9,8,1,0,2,6,3,3,5,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521605,1,0,0,0,0,3,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0
521606,0,0,5,0,1,1,0,3,1,1,0,0,0,2,1,0,0,1,0,2,0,0,2
521607,0,0,2,0,1,1,0,2,2,0,1,1,0,1,1,0,2,1,0,1,1,0,0


In [ ]:
df_sub2 = df_sub2.join(key_one_hot,on='userid')
df_sub2

,userid,grass_date,order_count,total_amount,category_encoded,purchase_num,order_count_sum,total_amount_sum,category_encoded_nunique,category_encoded_1,category_encoded_2,category_encoded_3,category_encoded_4,category_encoded_5,category_encoded_6,category_encoded_7,category_encoded_8,category_encoded_9,category_encoded_10,category_encoded_11,category_encoded_12,category_encoded_13,category_encoded_14,category_encoded_15,category_encoded_16,category_encoded_17,category_encoded_18,category_encoded_19,category_encoded_20,category_encoded_21,category_encoded_22,category_encoded_23
0,295790,2020-05-13,4,4,8,26,97,163,8,0,0,1,0,0,6,0,7,2,0,0,0,0,1,4,0,0,0,0,0,0,2,3
1,428982,2020-05-13,1,2,14,66,86,106,12,0,0,0,0,1,12,0,6,2,1,0,4,1,13,0,0,0,0,0,3,3,3,17
2,151685,2020-05-13,2,2,12,44,70,124,15,0,0,1,0,1,2,0,11,1,1,1,3,2,4,10,4,0,1,0,1,0,1,0
3,323635,2020-05-13,4,11,8,29,65,114,7,0,1,0,0,5,0,0,11,3,0,0,0,0,1,0,7,0,0,0,0,0,1,0
4,118610,2020-05-13,11,11,17,23,92,104,9,0,1,1,0,0,2,0,2,0,0,0,2,0,0,0,0,11,1,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7792951,275949,2020-03-26,1,1,8,30,62,93,11,1,0,2,0,1,7,0,12,1,0,0,2,1,0,0,0,0,1,0,0,1,1,0
7792952,255131,2020-03-26,1,1,20,30,56,77,9,0,2,0,0,0,12,0,0,0,1,1,2,0,0,0,0,0,0,0,2,1,1,8
7792953,319036,2020-03-26,1,1,15,19,80,129,6,0,0,0,0,0,1,0,14,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1
7792954,511880,2020-03-26,1,1,16,29,111,142,6,0,1,0,0,0,4,0,6,0,0,0,0,0,0,15,2,0,0,0,1,0,0,0


In [ ]:
df_sub2_unique = df_sub2[['userid','purchase_num','order_count_sum','total_amount_sum','category_encoded_nunique']].drop_duplicates().merge(key_one_hot,how='left',left_on='userid',right_on='userid')
df_sub2_unique

,userid,purchase_num,order_count_sum,total_amount_sum,category_encoded_nunique,category_encoded_1,category_encoded_2,category_encoded_3,category_encoded_4,category_encoded_5,category_encoded_6,category_encoded_7,category_encoded_8,category_encoded_9,category_encoded_10,category_encoded_11,category_encoded_12,category_encoded_13,category_encoded_14,category_encoded_15,category_encoded_16,category_encoded_17,category_encoded_18,category_encoded_19,category_encoded_20,category_encoded_21,category_encoded_22,category_encoded_23
0,295790,26,97,163,8,0,0,1,0,0,6,0,7,2,0,0,0,0,1,4,0,0,0,0,0,0,2,3
1,428982,66,86,106,12,0,0,0,0,1,12,0,6,2,1,0,4,1,13,0,0,0,0,0,3,3,3,17
2,151685,44,70,124,15,0,0,1,0,1,2,0,11,1,1,1,3,2,4,10,4,0,1,0,1,0,1,0
3,323635,29,65,114,7,0,1,0,0,5,0,0,11,3,0,0,0,0,1,0,7,0,0,0,0,0,1,0
4,118610,23,92,104,9,0,1,1,0,0,2,0,2,0,0,0,2,0,0,0,0,11,1,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502152,260482,1,1,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
502153,461761,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
502154,43617,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
502155,313529,1,2,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
